In [2]:
%load_ext autoreload
%autoreload 2

# Create mllib pipeline
from dspipes import Pipelines, MllibPipelines, RumblePipelines

## Rumble Pipelines (write)

In [9]:
MODELS = ['logistic', 'RandomForest', 'LinearSVC', 'NB']
ALL_PIPELINES = [0, 1, 3, 5]
DATASET = 'Criteo'
for m in MODELS:
    for i in ALL_PIPELINES:
        print(m, i)
        program = RumblePipelines.create_rumble_program(f"pipe_{i}", clf_mode=f"{m}")
        f = open(f"./rumble_experiment_scripts/query_{i}_{m}_{DATASET}.rumble", "w")
        f.write(program)
        f.close()




logistic 0
logistic 1
logistic 3
logistic 5
RandomForest 0
RandomForest 1
RandomForest 3
RandomForest 5
LinearSVC 0
LinearSVC 1
LinearSVC 3
LinearSVC 5
NB 0
NB 1
NB 3
NB 5


In [12]:
MODELS = ['logistic', 'RandomForest', 'LinearSVC', 'NB']
ALL_PIPELINES = [0, 1, 3, 5]
DATASET = 'Criteo'
f = open(f"run_rumble_experiments.sh", "a")
for m in MODELS:
    for i in ALL_PIPELINES:
        res = f"time spark-submit --num-executors 3 --executor-cores 1 --executor-memory 19g rumbledb-1.16.0.jar --query-path './rumble_experiment_scripts/query_{i}_{m}_{DATASET}.rumble'\n"
        f.write(res)
f.close() 


In [3]:
program = RumblePipelines.create_rumble_program("pipe_1")
print(program)

%%rumble
declare type local:mytype as {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"};
let $training-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
    }
let $test-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
    {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
    {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
    }
let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ "col1", "col2", "col3" ], "outputCol" : "features"})
let $scaler := get-estimator("MaxAbsScaler", {"inputCol": "features", "outputCol": "transformedFeatures"})
let $logisticregression := get-estimator("LogisticRe

In [4]:
program = RumblePipelines.create_rumble_program("pipe_3")
print(program)

%%rumble
declare type local:mytype as {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"};
let $training-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
    }
let $test-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
    {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
    {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
    }
let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ "col1", "col2", "col3" ], "outputCol" : "features"})
let $pca := get-estimator("PCA", {"inputCol": "features", "outputCol": "transformedFeatures", "k": 2})
let $logisticregression := get-estimator("LogisticRegres

In [5]:
program = RumblePipelines.create_rumble_program("pipe_5", clf_mode='NB')
print(program)

%%rumble
declare type local:mytype as {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"};
let $training-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
    }
let $test-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
    {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
    {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
    }
let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ "col1", "col2", "col3" ], "outputCol" : "features"})
let $pca := get-estimator("PCA", {"inputCol": "features", "outputCol": "pca_1_output", "k": 2})
let $pca2 := get-estimator("PCA", {"inputCol": "features", "out

## MLlib Pipelines

In [6]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext("local", "First App")
spark = SparkSession(sc)

In [10]:
# Define data
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

In [11]:
#0,1,3,5
pipeline = MllibPipelines.create_numerical_pipeline('pipe_0', imputer=False)
print(pipeline.extractParamMap())

{Param(parent='Pipeline_400199c7d508', name='stages', doc='a list of pipeline stages'): [LogisticRegression_2944e69ebd33]}


In [12]:
model = pipeline.fit(training)

In [15]:
prediction = model.transform(test)
result = prediction.select("features", "label", "probability", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.probability, row.prediction))

features=[-1.0,1.5,1.3], label=1.0 -> prob=[1.5982302680575156e-19,1.0], prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> prob=[0.9999999999983489,1.6511236822225328e-12], prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> prob=[9.067197386387449e-19,1.0], prediction=1.0
